In [77]:
#!/usr/bin/env python

from __future__ import print_function
import struct
import io

try:
    range = xrange
except NameError:
    pass


def _left_rotate(n, b):
    """Left rotate a 32-bit integer n by b bits."""
    return ((n << b) | (n >> (32 - b))) & 0xffffffff

In [86]:
def _process_chunk(chunk, h0, h1, h2, h3, h4):
    """Process a chunk of data and return the new digest variables."""
    assert len(chunk) == 64

    w = [0] * 80

    # Break chunk into sixteen 4-byte big-endian words w[i]
    for i in range(16):
        w[i] = struct.unpack(b'>I', chunk[i * 4:i * 4 + 4])[0]

    # Extend the sixteen 4-byte words into eighty 4-byte words
    for i in range(16, 80):
        w[i] = _left_rotate(w[i - 3] ^ w[i - 8] ^ w[i - 14] ^ w[i - 16], 1)

    for i in range(0, 80):
        print('{:x}'.format(w[i]));
    # Initialize hash value for this chunk
    a = h0
    b = h1
    c = h2
    d = h3
    e = h4

    for i in range(80):
        if 0 <= i <= 19:
            # Use alternative 1 for f from FIPS PB 180-1 to avoid bitwise not
            f = d ^ (b & (c ^ d))
            k = 0x5A827999
        elif 20 <= i <= 39:
            f = b ^ c ^ d
            k = 0x6ED9EBA1
        elif 40 <= i <= 59:
            f = (b & c) | (b & d) | (c & d)
            k = 0x8F1BBCDC
        elif 60 <= i <= 79:
            f = b ^ c ^ d
            k = 0xCA62C1D6
        
        print('{:08x},{:08x},{:08x},{:08x},{:08x},{:08x},{:08x}'.format(a,b,c,d,e,f,w[i]))
        print ('{:08x}'.format(_left_rotate(a, 5)))
        a, b, c, d, e = ((_left_rotate(a, 5) + f + e + k + w[i]) & 0xffffffff, a, _left_rotate(b, 30), c, d)
        
        print('{:08x},{:08x},{:08x},{:08x},{:08x}'.format(a,b,c,d,e))
        
    # Add this chunk's hash to result so far
    h0 = (h0 + a) & 0xffffffff
    h1 = (h1 + b) & 0xffffffff
    h2 = (h2 + c) & 0xffffffff
    h3 = (h3 + d) & 0xffffffff
    h4 = (h4 + e) & 0xffffffff
    return h0, h1, h2, h3, h4

In [87]:
class Sha1Hash(object):
    """A class that mimics that hashlib api and implements the SHA-1 algorithm."""

    name = 'python-sha1'
    digest_size = 20
    block_size = 64

    def __init__(self):
        # Initial digest variables
        self._h = (
            0x67452301,
            0xEFCDAB89,
            0x98BADCFE,
            0x10325476,
            0xC3D2E1F0,
        )

        # bytes object with 0 <= len < 64 used to store the end of the message
        # if the message length is not congruent to 64
        self._unprocessed = b''
        # Length in bytes of all data that has been processed so far
        self._message_byte_length = 0

    def update(self, arg):
        """Update the current digest.

        This may be called repeatedly, even after calling digest or hexdigest.

        Arguments:
            arg: bytes, bytearray, or BytesIO object to read from.
        """
        if isinstance(arg, (bytes, bytearray)):
            arg = io.BytesIO(arg)

        # Try to build a chunk out of the unprocessed data, if any
        chunk = self._unprocessed + arg.read(64 - len(self._unprocessed))

        # Read the rest of the data, 64 bytes at a time
        while len(chunk) == 64:
            self._h = _process_chunk(chunk, *self._h)
            self._message_byte_length += 64
            chunk = arg.read(64)

        self._unprocessed = chunk
        return self

    def digest(self):
        """Produce the final hash value (big-endian) as a bytes object"""
        return b''.join(struct.pack(b'>I', h) for h in self._produce_digest())

    def hexdigest(self):
        """Produce the final hash value (big-endian) as a hex string"""
        return '%08x%08x%08x%08x%08x' % self._produce_digest()

    def _produce_digest(self):
        """Return finalized digest variables for the data processed so far."""
        # Pre-processing:
        message = self._unprocessed
        message_byte_length = self._message_byte_length + len(message)

        # append the bit '1' to the message
        message += b'\x80'

        # append 0 <= k < 512 bits '0', so that the resulting message length (in bytes)
        # is congruent to 56 (mod 64)
        message += b'\x00' * ((56 - (message_byte_length + 1) % 64) % 64)

        # append length of message (before pre-processing), in bits, as 64-bit big-endian integer
        message_bit_length = message_byte_length * 8
        message += struct.pack(b'>Q', message_bit_length)

        # Process the final chunk
        # At this point, the length of the message is either 64 or 128 bytes.
        h = _process_chunk(message[:64], *self._h)
        if len(message) == 64:
            return h
        return _process_chunk(message[64:], *h)

In [88]:
def sha1(data):
    """SHA-1 Hashing Function

    A custom SHA-1 hashing function implemented entirely in Python.

    Arguments:
        data: A bytes or BytesIO object containing the input message to hash.

    Returns:
        A hex SHA-1 digest of the input message.
    """
    return Sha1Hash().update(data).hexdigest()

In [89]:
print('sha1-digest:', sha1(''))

80000000
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
2
0
0
4
0
2
8
0
0
10
0
a
20
6
0
40
8
28
80
8
0
108
0
a0
200
64
0
408
88
29c
800
80
28
1088
0
a40
2000
668
80
4088
880
28c8
8000
8a8
80
108e8
0
a000
20080
6400
0
40800
8800
29c10
80000
8080
2820
1088c0
0
a40c0
200080
67452301,efcdab89,98badcfe,10325476,c3d2e1f0,98badcfe,80000000
e8a4602c
1fb498b3,67452301,7bf36ae2,98badcfe,10325476
1fb498b3,67452301,7bf36ae2,98badcfe,10325476,fbfbfefe,00000000
f6931663
5d43e370,1fb498b3,59d148c0,7bf36ae2,98badcfe
5d43e370,1fb498b3,59d148c0,7bf36ae2,98badcfe,79d36ac0,00000000
a87c6e0b
158d2f62,5d43e370,c7ed262c,59d148c0,7bf36ae2
158d2f62,5d43e370,c7ed262c,59d148c0,7bf36ae2,45d12aa0,00000000
b1a5ec42
cdecfb5d,158d2f62,1750f8dc,c7ed262c,59d148c0
cdecfb5d,158d2f62,1750f8dc,c7ed262c,59d148c0,d760284c,00000000
bd9f6bb9
4953565e,cdecfb5d,85634bd8,1750f8dc,c7ed262c
4953565e,cdecfb5d,85634bd8,1750f8dc,c7ed262c,97704bd8,00000000
2a6acbc9
e44ab766,4953565e,737b3ed7,85634bd8,1750f8dc
e44ab766,4953565e,737b3ed7,85634bd8,1